# Wind Downscaling

## Prerequisites

* Conda environment
* Get a Copernicus API key from: https://cds.climate.copernicus.eu/api-how-to
  * create a file at \$HOME/.cdsapirc with the required UID and key

## Install the required packages

In [ ]:
import os
from datetime import date, datetime
from pathlib import Path

In [ ]:
!pip uninstall downscaling -y
!pip install -U git+https://github.com/OpheliaMiralles/wind-downscaling-gan.git

In [ ]:
!conda install -y -c conda-forge gdal tensorflow xarray numpy=1.19.5 pandas pysftp cdsapi elevation rasterio dask python-dotenv

In [ ]:
!pip install topo-descriptors

## Set configuration

In [ ]:
DATA_ROOT = Path('./data')
ERA5_DATA_FOLDER = DATA_ROOT / 'ERA5_FR'
DEM_DATA_FILE = DATA_ROOT / 'dem/France-90m-DEM.tif'
START_DATE = date(2016,4,1)
END_DATE = date(2016,4,2)
LONGITUDE_R = (-4.96, 8.3)
LATITUDE_R = (42.2, 51.3)

## Data Loading

In [ ]:
from downscaling.data import download_ERA5
download_ERA5(ERA5_DATA_FOLDER, START_DATE, END_DATE, LATITUDE_R, LONGITUDE_R)
print('Done')

In [ ]:
if not DEM_DATA_FILE.exists():
    dest = str(DEM_DATA_FILE)
    !eio --product SRTM3 clip -o {dest} --bounds -4.96 42.2 8.3 51.3

## Downscaling

In [ ]:
from downscaling import downscale, plot_wind_fields
import xarray as xr

In [ ]:
era5 = xr.open_mfdataset(ERA5_DATA_FOLDER.glob('20160401*surface*.nc'))
raster_topo = xr.open_rasterio(DEM_DATA_FILE)
downscaled_maps = downscale(era5, raster_topo, range_lon=LONGITUDE_R, range_lat=LATITUDE_R)

In [ ]:
fig = plot_wind_fields(downscaled_maps.isel(time=0), range_lon=LONGITUDE_R, range_lat=LATITUDE_R)
fig.show()

## Downscaling wind fields in Switzerland with target COSMO-1

In [ ]:
from downscaling import process_era5, process_topo, predict

In [ ]:
ERA5_DATA_FOLDER = DATA_ROOT / 'ERA5'
DEM_DATA_FILE = DATA_ROOT / 'dem/Switzerland-90m-DEM.tif'
START_DATE = date(2016,4,1)
END_DATE = date(2016,4,1)
LONGITUDE_R = (5.8, 10.6)
LATITUDE_R = (45.75, 47.9)

In [ ]:
import requests
r = requests.get('https://github.com/OpheliaMiralles/wind-downscaling-gan/blob/master/src/downscaling/switzerland_cosmo_map.nc')
with open('switzerland_cosmo_map.nc', 'wb') as f:
    f.write(r.content)
HIGH_RES_TEMPLATE = xr.open_dataset('switzerland_cosmo_map.nc')

In [ ]:
era5 = process_era5(xr.open_dataset(ERA5_DATA_FOLDER / '20171005_era5_surface_hourly.nc'), high_res_template=HIGH_RES_TEMPLATE)
topo = process_topo(xr.open_rasterio(DEM_DATA_FILE), high_res_template=HIGH_RES_TEMPLATE)

In [ ]:
downscaled_maps = predict(era5, topo, high_res_template=HIGH_RES_TEMPLATE)

## Plots of wind fields in Switzerland

In [ ]:
from downscaling import plot_elevation, plot_wind_fields
from cartopy.crs import epsg
HIGH_RES_CRS = epsg(21781)  # crs for the target map, for plots

In [ ]:
raw_topo = xr.open_rasterio(DEM_DATA_FILE)
fig_topo = plot_elevation(raw_topo, range_lon=LONGITUDE_R, range_lat=LATITUDE_R)
fig_topo.show()

In [ ]:
fig = plot_wind_fields(downscaled_maps.isel(time=0), range_lon=LONGITUDE_R, range_lat=LATITUDE_R, high_res_crs=HIGH_RES_CRS)
fig.show()